In [ ]:
import sys
sys.path.append('../')
from examples.tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import logging
import re
import datetime

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")

In [ ]:
IMG_W = 32
IMG_H = 32
IMG_C = 1

last_epoch = 0
batch_size = 128
latent_dim = 128
RUNTIME = "cyclegan"
SAMPLES_PATH = f"samples/{RUNTIME}"
CHECKPOINT_PATH = f"model_checkpoints/{RUNTIME}"
LOG_DIR = f"/qarr/studia/magister/models/{RUNTIME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
try:
    checkpoints = os.listdir(CHECKPOINT_PATH)
    last_epoch = np.max([int(re.search(r"-[0-9]+\.", i)[0][1:-1]) for i in checkpoints if
                             re.search(r"-[0-9]+\.", i)])
    print("Detected {} epoch as last checkpoint".format(last_epoch))
except (ValueError,  FileNotFoundError):
    last_epoch = 0
    print("Did not detect any checkpoints to continue from")

# Model

In [ ]:
pix2pix.unet_generator(1, norm_type="instancenorm")

In [ ]:
def unet_generator_mnist():
    ...

# Cycle loss

In [ ]:
LAMBDA = 10
base_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(generated):
    return base_loss(tf.ones_like(generated), generated)


def discriminator_loss(real, generated):
  real_loss = base_loss(tf.ones_like(real), real)

  generated_loss = base_loss(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

return total_disc_loss * 0.5


def cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image)) # try with squared?

  return LAMBDA * loss1


def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.

    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)
    
    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)

    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)

  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)

  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))

  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))
